In [84]:
import pandas as pd
import numpy
import matplotlib
import os
import subprocess

In [85]:
path='C:/Users/asd/Desktop/500Frames'

In [86]:
totalFrames=0
totalID=0
videoNo=0
for i in range(1,17):
    inpath=path+"/"+str(i)
    for j in range(1,8):
        try:
            fullPath=inpath+"/"+str(j)+"/00"+str(j)+".csv"
            #print(fullPath)
            readFile=pd.read_csv(fullPath)
            totalFrames=totalFrames + max(readFile.frameNumber)
            ids=readFile.id
            idSet=set(ids)
            totalID=totalID+len(idSet)
            videoNo=videoNo+1
        except:
            videoNo=videoNo-1
            print("file not found")

file not found
file not found
file not found
file not found
file not found
file not found
file not found
file not found
file not found
file not found
file not found
file not found
file not found
file not found
file not found
file not found


In [87]:
print(totalFrames)
print(totalID)
print(videoNo)

47809
2257
80


In [88]:
print(readFile)

      frameNumber          label  id  x_TL  y_TL  x_BR  y_BR  isOccluded  \
0               0            Bus   0     1   300   290     1           1   
1               0    Two-Wheeler   1   509   173   526   141           0   
2               0  Auto-Rickshaw   2   559   180   596   139           0   
3               0    Two-Wheeler   3   622   166   633   141           0   
4               0            Car   4   647   166   670   145           0   
5               1  Auto-Rickshaw   2   558   179   595   138           0   
6               1            Car   4   647   165   670   144           0   
7               1    Two-Wheeler   1   506   172   523   141           0   
8               1            Bus   0     1   300   293     1           1   
9               1    Two-Wheeler   3   621   165   632   140           0   
10              2  Auto-Rickshaw   2   558   179   595   138           0   
11              2            Car   4   647   165   670   144           0   
12          

In [89]:
labels=readFile.label
ids=readFile.id
lines=len(readFile)
idList=[]
labelList=[]
for i in range(lines):
    if ids[i] in idList:
        continue
    else:
        idList.append(ids[i])
        labelList.append(labels[i])

In [90]:
print(idList)
print(labelList)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 11, 13, 15]
['Bus', 'Two-Wheeler', 'Auto-Rickshaw', 'Two-Wheeler', 'Car', 'Car', 'Car', 'Two-Wheeler', 'Two-Wheeler', 'Auto-Rickshaw', 'Pedestrian', 'Two-Wheeler']


### Total Number of bounding boxes for each class and total along with occlusion information

In [110]:
totalbbox=0
totalPeds=0
totalBus=0
totalTwo=0
totalCar=0
totalTruck=0
totalAuto=0
irrcount=0
allClasses=[]
ijkPedCross=[]
ijkPark=[]
ijkStop=[]
ijkRight=[]
ijkU=[]
for i in range(1,17):
    inpath=path+"/"+str(i)
    for j in range(1,8):
        try:
            fullPath=inpath+"/"+str(j)+"/00"+str(j)+".csv"
            readFile=pd.read_csv(fullPath)
            totalbbox=totalbbox+len(readFile)
            
            
            
            
            
            
            
            
            
            
            
            
            
            
            
            
            for k in range(len(readFile)):
                if(not(readFile.label[k] in allClasses)):
                    allClasses.append(readFile.label[k])
                if(readFile.label[k]=='Pedestrian Crossing'):
                    irrcount=irrcount+1
                    ijkPedCross.append(str(i)+"-"+str(j)+"-"+str(k))
                if(readFile.label[k]=='No Parking'):
                    irrcount=irrcount+1
                    ijkPark.append(str(i)+"-"+str(j)+"-"+str(k))
                if(readFile.label[k]=='No Stopping'):
                    irrcount=irrcount+1
                    ijkStop.append(str(i)+"-"+str(j)+"-"+str(k))
                if(readFile.label[k]=='Right Turn'):
                    irrcount=irrcount+1
                    ijkRight.append(str(i)+"-"+str(j)+"-"+str(k))
                if(readFile.label[k]=='U-Turn'):
                    irrcount=irrcount+1
                    ijkU.append(str(i)+"-"+str(j)+"-"+str(k))
        except:
             print("file not found",i,j)


file not found 1 1
file not found 3 7
file not found 4 4
file not found 4 5
file not found 4 6
file not found 14 5
file not found 14 6
file not found 14 7
file not found 15 3
file not found 15 4
file not found 15 5
file not found 15 6
file not found 15 7
file not found 16 5
file not found 16 6
file not found 16 7


In [112]:
print(totalbbox)
print(allClasses)
print(irrcount)
print(ijkPedCross)
print(ijkPark)
print(ijkStop)
print(ijkRight)
print(ijkU)

221350
['Car', 'Two-Wheeler', 'Pedestrian', 'Auto-Rickshaw', 'Truck', 'Cyclist', 'Bus', 'Animal', 'Pedestrian Crossing', 'No Parking', 'No Stopping', 'Right Turn', 'U-Turn']
245
['5-7-2193', '5-7-2196', '5-7-2200', '5-7-2204', '5-7-2208', '5-7-2211', '5-7-2215', '5-7-2219', '5-7-2223', '5-7-2227', '5-7-2231', '5-7-2235', '5-7-2239', '5-7-2243', '5-7-2247', '5-7-2251', '5-7-2255', '5-7-2259', '5-7-2263', '5-7-2267', '5-7-2271', '5-7-2275', '5-7-2279', '5-7-2283', '5-7-2287', '5-7-2292', '5-7-2296', '5-7-2300', '5-7-2303', '5-7-2307', '5-7-2311', '5-7-2315', '5-7-2321', '5-7-2326', '5-7-2330', '5-7-2336', '5-7-2340', '5-7-2345', '5-7-2350', '5-7-2355', '5-7-2361', '5-7-2365', '5-7-2371', '5-7-2376', '5-7-2380', '5-7-2387', '5-7-2392', '5-7-2395', '5-7-2402', '5-7-2405', '8-7-2073', '8-7-2078', '8-7-2083', '8-7-2088', '8-7-2093', '8-7-2099', '8-7-2105', '8-7-2110', '8-7-2115', '8-7-2118', '8-7-2125']
['6-7-449', '6-7-454', '6-7-459', '6-7-464', '6-7-469', '6-7-474', '6-7-479', '6-7-484', 

In [93]:
ids=readFile.id
idSet=set(ids)
print(idSet)
print(len(idSet))

{0, 1, 2, 3, 4, 5, 6, 7, 8, 11, 13, 15}
12
